<a href="https://colab.research.google.com/github/boorlakov/linear_miniproject_funcan_appr/blob/main/linear_miniproject_funcan_appr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Мини-проект по функану. Кусочно-линейные функции.
---
### Программа по заданной сетке с файла строит приближение функции кусочно-линейным базисом.
### Для запуска программы **последовательно** нажимайте туда, куда **указывает** красная стрелочка. Также вы можете запустить код ячейки путём нажатия комбинации клавиш ```Ctrl+Enter```.
### Ввод сетки происходит с **файла**, файл должен быть сформирован **заранее**.
### Формат файла - ```.txt```
### Формат записи сетки должен быть в следующем виде: 
 
```
0.0, 1.0, 2.0
```
Разделителем является ```, ``` (запятая пробел)
 
---
При дроблении сетки несколько раз **запустите** необходимую ячейку **нужное** количество раз.
 
Если хотите изменить начальную сетку, то вы можете это сделать **двумя** способами:
 
1) Открыть загруженный файл в Google Colab и изменить данные в нем, для этого нужно:
    
    1.1) Открыть папку "Файлы" - иконка папки в меню слева
    
    1.2) Найти загруженный файл и кликнуть на него двойным щелчком левой кнопки мыши
 
    1.3) Изменить входные данные
 
2) Удалить загруженный файл и загрузить файл с новыми данными еще раз:
    
    2.1) Открыть папку "Файлы" - иконка папки в меню слева
    
    2.2) Найти загруженный файл и кликнуть на него правой кнопкой мыши
 
    2.3) Нажать `Удалить файл`
 
    2.4) Перейти в ячейку (2) и заново загрузить файл
 
Не забывайте инициализировать сетку с файла: для этого запустите код в ячейке **(5)**. Удостоверьтесь, что данные с файла **соответствуют** начальной сетке.

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (1) Устанавливаем все необходимые компоненты</font></b>
 
import math
 
import os
import io
import sys
 
from matplotlib import pyplot as plt
import numpy as np
 
from google.colab import files
import warnings
 
modulename = 'datetime'
if modulename not in sys.modules:
    print('ERR: You have not imported the {} module'.format(modulename))
else:
    print('Всё хорошо, можно идти дальше.')

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (2) Загружаем входные данные с файла</font></b>
 
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
os.rename(fn, fn.replace(" ", ""))
fn = fn.replace(" ", "")
print('Файл успешно загружен. Можно идти дальше.')

##**(3)** **Введите** функцию для приближения и **запустите код** в  **следующей** ячейке.


Пока что кусочно-заданные функции **недоступны** для использования. Для теста кейс функций используйте $ \left|x\right| $

In [ ]:
# (4) Когда вы ввели функцию, запустите код в ячейке
def target_func( x ):
    return x**6

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (5) Инициализировать начальную сетку</font></b>

num_of_test = 0
data_to_output = ''

dt = np.dtype( np.float64 )
grid = np.fromfile( fn, dtype=dt, sep=',' )
 
print( 'Начальная сетка:', grid )
 
ax1 = plt.axes( frameon=False )
ax1.set_frame_on( False )
ax1.get_xaxis().tick_bottom()
ax1.axes.get_yaxis().set_visible( False )
 
plt.hlines( 1, grid[0], grid[len(grid)-1] )  # Draw a horizontal line
plt.eventplot( grid, orientation='horizontal', colors='b' )
 
plt.axis()
plt.title( "Начальная сетка" )
plt.show()

In [ ]:
#@{ display-mode: "form" }
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (6) Выбираем то, как будем дробить сетку (пропустите эту ячейку, если не хотитие дробить сетку)</font></b>
 
def change_grid( grid, mode, num, start_node=0, end_node=0 ):
    """Changes the grid by slices.
    
    Parameters
    ----------
    grid: numpy array
        sequantually stored points in array
    mode: python string
        fully or noded
    num: python int
        number into how many parts grid will be divided
    start_node: python int, optional
        start node is starting point of dividing
        by default start node is 0
    end_node: python int, optional
        end node is ending point of dividing
        by default end node is 0
    
    Returns
    -------
    grid
        a numpy array
    """
    
    # making grid by a string
    tmp_grid = str( grid[ 0 ] ) + ' '
    #Режим полностью
    FULLY_RUS = '\u043F\u043E\u043B\u043D\u043E\u0441\u0442\u044C\u044E'
    if mode == FULLY_RUS: 
        for i in range( 1, len( grid ) ):
            step = (grid[ i ] - grid[ i-1 ]) / num
            tmp_val = grid[ i-1 ]
            for j in range( 0, num-1 ):
                tmp_val += step
                tmp_grid += str( tmp_val ) + ' '
    
            tmp_grid += str( grid[ i ] ) + ' '
    #Узловой режим
    else:
        for i in range( 1, start_node ):
            tmp_grid += str( grid[i] ) + ' '
    
        step = ( grid[ end_node ] - grid[ start_node ] ) / num
        tmp_val = grid[ start_node ]
    
        if start_node != 0:
            tmp_grid += str( tmp_val ) + ' '
    
        for j in range( 0, num-1 ):
            tmp_val += step
            tmp_grid += str( tmp_val ) + ' '
        for i in range( end_node, len( grid ) ):
            tmp_grid += str( grid[i] ) + ' '
    
    grid = np.fromstring( tmp_grid, dtype=dt, sep=' ' )
    
    return grid 
 
 
Режим = '\u0443\u0437\u043B\u043E\u0432\u043E\u0439' #@param ["узловой", "полностью"]
Промежуток = 2#@param {type:"number"}
Количество =  3#@param {type:"number"}
 
mode = Режим
start_node = Промежуток - 1
end_node = Промежуток
num = Количество
 
FULLY_RUS = '\u043F\u043E\u043B\u043D\u043E\u0441\u0442\u044C\u044E'
if ( end_node >= len( grid ) and
     mode == FULLY_RUS ):
    print('Ошибка ввода данных: промежуток за пределом массива')
 
grid = change_grid( grid, mode, num, start_node, end_node )
 
print( 'Измененная сетка:', grid )
 
ax1 = plt.axes( frameon=False )
ax1.set_frame_on( False )
ax1.get_xaxis().tick_bottom()
ax1.axes.get_yaxis().set_visible( False ) 
 
plt.hlines( 1, grid[ 0 ], grid[ len( grid )-1 ] )  # Draw a horizontal line
plt.eventplot( grid, orientation='horizontal', colors='b' )
 
plt.axis()
plt.title( "Изменённая сетка" )
plt.show()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (7) Бим-бим, бам-бам, и всё готово!</font></b>
 
 
 
#--- Start of functions definitions
 
 
def L2_inner_product( f, g, dx ):
    """Returns a inner product of given functions in L2 space. 
    Calculating integral from a to b of f(x) * g(x) dx using trapezoidal rule
 
    Parameters
    ----------
    f : python function
        Notice, that f must include linspace X
    g : python function
        Notice, that g must include linspace X
    dx : float
        Do not forget, that integration boundaries matter 
        Like dx = ( b-a ) * np.pi / ( n-1 )
 
    Returns
    -------
    float
      L2 inner product value
    """
 
    return np.trapz( f*g, dx=dx ) / np.pi
 
 
def node_func( x, start_node, end_node, mode ):
    """Returns a value of node function (linear)
    
    Parameters
    ----------
    x : lin space
        Given argument to the approximating function, 
        working this function as a array
    start_node : int
        Gtart node is starting point on grid
    end_node : int
        End node is ending point on grid
    mode : int, because of easy to use
        1 - is node function increasing
        2 or anything else - is node function decreasing
 
    Returns
    -------
    float
        Value of node function (linear)
    """
 
    if mode == 1:
        # x   - x_(i-1)
        # -------------
        # x_i - x_(i-1)
        return ( x - start_node ) / ( end_node - start_node )
    else:
        # x_i - x
        # -------------
        # x_i - x_(i-1)
        return ( end_node - x ) / ( end_node - start_node )
 
 
def diff_func( f, g ):
    """Returns a difference between given functions
 
    Parameters
    ----------
    f : python function
    g : python function
 
    Returns
    -------
    function
        function of difference between given functions
    """
 
    return f - g
 
 
def appr_func_noded( x, 
                     weight_coef_start,
                     weight_coef_end,
                     start_node,
                     end_node ):
    """Approximating function at the node
 
    Parameters
    ----------
    x : lin space or float
        given argument to the approximating function
    weight_coef_start : int
        weight coeficient from solving the matrix equation (q1)
    weight_coef_end : int
        weight coeficient from solving the matrix equation (q2)
    start_node : int
        start node is starting point on grid
    end_node : int
        end node is ending point on grid
    
    Returns
    -------
    int
        value of approximating function at the node
    """
 
    # q2 - q1
    # ------- X + q1*x2 - q2*x1
    # x2 - x1
    return ( ( ( weight_coef_end   - weight_coef_start ) * x 
            + (   weight_coef_start * end_node - weight_coef_end * start_node ) ) 
            / (   end_node - start_node ))
    
 
def coef_x1_appr_func_noded( x,
                             weight_coef_start,
                             weight_coef_end,
                             start_node,
                             end_node ):
    return ( ( weight_coef_end - weight_coef_start ) /
             ( end_node - start_node ) )
 
 
def coef_x0_appr_func_noded( x,
                             weight_coef_start,
                             weight_coef_end,
                             start_node,
                             end_node ):
    return ( ( weight_coef_start * end_node - weight_coef_end * start_node) /
             ( end_node - start_node ) )
 
 
def appr_func_to_str( n, weight_coefs, grid ):
    """Make a approximating function at the node as a string
 
    Parameters
    ----------
    n : int
        Number of samples to generate. Must be non-negative.
    weight_coefs : numpy array
        Vector q_i from solving the matrix equation.
    grid : numpy array
        grid of points of appromaximation
    
    Returns
    -------
    appr_func_str : str
        string view of appromaximating case function 
    """
 
    X = np.linspace( grid[ 0 ], grid[ 1 ], n )
    appr_func_str = 'u(x) = ' + '\n '
    coef_x1 = coef_x1_appr_func_noded( X,
                                       weight_coefs[ 0 ],
                                       weight_coefs[ 1 ],
                                       grid[ 0 ],
                                       grid[ 1 ])
    coef_x0 = coef_x0_appr_func_noded( X,
                                       weight_coefs[ 0 ],
                                       weight_coefs[ 1 ],
                                       grid[ 0 ],
                                       grid[ 1 ] )
    
    appr_func_str += ( '(' + str( coef_x1 ) + ')*x + (' +
                             str( coef_x0 ) + '), ' )
    
    appr_func_str += ( 'где ' + str( grid[ 0 ] ) +
                    ' <= x <= ' + str( grid[ 1 ] ) + '\n' )
    for i in range ( 2, len( grid ) ):
        X = np.linspace( grid[ i-1 ], grid[ i ], n )
        coef_x1 = coef_x1_appr_func_noded( X,
                                           weight_coefs[ i-1 ],
                                           weight_coefs[ i ],
                                           grid[ i-1 ],
                                           grid[ i ] )
        coef_x0 = coef_x0_appr_func_noded( X,
                                           weight_coefs[ i-1 ],
                                           weight_coefs[ i ],
                                           grid[ i-1 ],
                                           grid[ i ] )
        
        appr_func_str += ' (' + str( coef_x1 )
        appr_func_str += ')*x + (' + str( coef_x0 ) + '), '
    
        appr_func_str += ( 'где ' + str( grid[ i-1 ] ) + 
                            ' <= x <= ' + str( grid[ i ] ) + '\n' )
    return appr_func_str
 
#--- End of functions definitions
 
 
 
n = 10000000
# By default 1.0 is starting lenght of integration bounds
dx = 1.0 * np.pi / ( n-1 )
# linspace will be changed to calculate integrals correctly
X = np.linspace( 0, 1, n )
 
# Making local matrix to calculate global matrix of appromaximation  
nodes_length_tmp = ''
for i in range( 1, len( grid ) ):
    nodes_length_tmp += str( grid[ i ] - grid[ i-1 ] ) + ' '
nodes_length = np.fromstring( nodes_length_tmp, dtype=dt, sep=' ' )
 
base_local_mat = np.zeros( ( 2, 2 ) )
#1 1 element of local matrix
base_local_mat[ 0 ][ 0 ] = 1/3.
 
#1 2 element of local matrix
base_local_mat[ 0 ][ 1 ] = 1/6.
 
#2 1 element of local matrix
base_local_mat[ 1 ][ 0 ] = 1/6.
 
#2 2 element of local matrix
base_local_mat[ 1 ][ 1 ] = 1/3.
 
# Preparation of calculation global matrix
global_matrix = np.zeros( ( len( grid ), len( grid ) ) )
 
for k in range( 0, len( nodes_length ) ):
  for i in range( k, len( base_local_mat ) + k ):
    for j in range( k, len( base_local_mat ) + k ):
      global_matrix[ i ][ j ] += nodes_length[ k ] * base_local_mat[ i%2 ][ j%2 ]
 
# To calculate weight coeficients prepare right side of matrix equation 
L2_inner_products = np.zeros( len( grid ) )
for i in range( 1, len( grid ) ):
    X = np.linspace( grid[ i-1 ], grid[ i ], n )
    dx = nodes_length[ i-1 ] * np.pi / ( n-1 )
    for j in range( 0, 2 ):
        L2_inner_products[ i-1+j ] += L2_inner_product( target_func( X ),
                                                        node_func( X,
                                                                grid[ i-1 ],
                                                                grid[ i ],
                                                                j ),
                                                        dx )
 
weight_coefs = np.linalg.solve( global_matrix, L2_inner_products )
 
# Calculating the measurement error of our linear appromaximation
norm = 0.0
for i in range ( 1, len( grid ) ):
    X = np.linspace( grid[ i-1 ], grid[ i ], n )
    dx = nodes_length[ i-1 ] * np.pi / ( n-1 )
    norm += 1 * L2_inner_product( diff_func( target_func( X ), 
                                             appr_func_noded( X, 
                                                             weight_coefs[ i-1 ], 
                                                             weight_coefs[ i ], 
                                                             grid[ i-1 ], 
                                                             grid[ i ])), 
                                    diff_func( target_func( X ), 
                                               appr_func_noded( X,
                                                             weight_coefs[ i-1 ],
                                                             weight_coefs[ i ], 
                                                             grid[ i-1 ], 
                                                             grid[ i ] ) ),
                                    dx )
# Do not forget, that norm = sqrt(scalar_product)  
norm = norm**( 1/2 )
 
print( 'Вектор скалярных произведений:' )
print( L2_inner_products )
print( ' ' )
 
np.transpose( weight_coefs )
print( 'Решение СЛАУ (весовые коэффициенты):' )
print( weight_coefs )
print( ' ' )
 
print( '||f-u|| =', norm )  
print( ' ' )
u_x = appr_func_to_str( n, weight_coefs, grid )
# Printing the appromaximating function 
print( u_x )
 
 
num_of_test += 1
 
data_to_output += '\n' + '-----' + '\n\n'
data_to_output += 'Тест №' + str( num_of_test ) + '\n'
data_to_output += 'f(x) = ' + 'отредачить' + '\n'
data_to_output += 'Сетка = ' + str( grid ) + '\n\n'
data_to_output += 'Весовые коэффициенты = ' + str( weight_coefs ) + '\n\n'
data_to_output += '||f-u|| = ' + str(norm) + '\n\n'
data_to_output += u_x + '\n'
data_to_output += '-----' + '\n\n'

In [ ]:
#@{ display-mode: "form" }
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (8) Вычислить приближающую функцию в точке</font></b>
 
 
 
#--- Start of functions definitions
 
def appr_func_val(x0): 
    """Returns a value of approximate function of given x point.
 
    x0 : float number
    returns : value of approximate function of given x point
    """     
 
    #mb binary search is much better         
    for i in range (1, len(grid)):
        if (x0 <= grid[i] and x0 >= grid[i-1]):
            return appr_func_noded(x0, 
                        weight_coefs[i-1], 
                        weight_coefs[i], 
                        grid[i-1], 
                        grid[i]), weight_coefs[i-1], weight_coefs[i]
    return 'undefined'
    
#--- End of functions definitions 
 
 
 
Точка =  3#@param {type:"number"}
x0 = Точка
 
print('u(x0) = ', appr_func_val(x0)[0])
print('f(x0) = ', target_func(x0))

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (9) Вывести вектор скалярных произведений, СЛАУ и норму в Latex (можно пропустить) </font></b>
 
 
def bmatrix( a ):
    """
    Returns a LaTeX bmatrix
 
    a : numpy array
    returns : LaTeX bmatrix as a string
    """
    if len( a.shape ) > 2:
        raise ValueError( 'bmatrix can at most display two dimensions' )
    lines = str(a).replace( '[', '' ).replace( ']', '' ).splitlines()
    rv = [ r'\begin{bmatrix}' ]
    rv += [ '  ' + ' & '.join( l.split() ) + r'\\' for l in lines ]
    rv +=  [ r'\end{bmatrix}' ]
    return '\n'.join( rv )
 
 
def norm_tolatex( norm ):
    """
    Returns a LaTeX bmatrix
 
    norm : float number
    returns : LaTeX norm as a string
    """
    rv = [ r'\begin{equation*}' ]
    rv += '\n'
    rv += '{||\mathbf{f}_{}-\mathbf{u}_{}||} = '
    rv += str( norm ) + '\n'
    rv + [r'\end{equation*}']
    return ''.join( rv )
 
 
 
print( 'Пожалуйста, скопируйте результат работы программы в следующую ячейку.' )
print( 'Дважды нажав на ячейку, вы откроете ячейку для редактирования.' )
print( 'Рекомендуется копировать комбинацией Ctrl + C, а вставлять Ctrl + V.\n\n' )
 
print( 'Вектор скалярных произведений:' )
print( bmatrix(L2_inner_products) + '\n\n' )
 
print( 'Решение СЛАУ:' )
print( bmatrix(weight_coefs) + '\n\n' )
 
print( 'Норма: ' )
print( norm_tolatex(norm) + '\end{equation*}' )

## **(10) Редактировать следующую ячейку (дважды кликните и вставьте)**
## Можно пропустить

##**(Только если нужно в Latex) (11) Сюда вставить результат ячейки 9**


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (12) Рисуем графики (ALPHA, можно пропустить) </font></b>
y2_vals = np.zeros( len( grid ) )
 
y2_vals[ 0 ] = ( appr_func_noded( grid[ 0 ],
                                  weight_coefs[ 0 ],
                                  weight_coefs[ 1 ],
                                  grid[ 0 ],
                                  grid[ 1 ] ) ) 
for i in range ( 1, len( grid ) ):
    y2_vals[ i ] = ( appr_func_noded( grid[ i ],
                                      weight_coefs[ i-1 ],
                                      weight_coefs[ i ],
                                      grid[ i-1 ],
                                      grid[ i ] ) )
 
n = np.shape( y2_vals )[ 0 ]
X = np.linspace( grid[ 0 ], grid[ len( grid )-1 ], len( grid ) )
 
n1 = 1000000
X1 = np.linspace( grid[ 0 ], grid[ len( grid )-1 ], n1 )
 
y_1 = target_func( X1 )
# setting the axes at the centre
fig = plt.figure()
 
# plot the function
plt.plot( X1, y_1, 'r', label='Приближаемая функция' )
plt.plot( X, y2_vals, label='Приближающая функция' )
plt.title( "Сравнительный график" )
 
plt.xlabel( "x" )
plt.ylabel( "y" )
plt.legend()
plt.show()

In [ ]:
#@title <b><font color="red" size="+3"></font><font color="black" size="+3"> (13) Для отладки отдельных компонент (TEST) </font></b>
n=10000000
X = np.linspace(0, np.pi, n)
dx = np.pi * np.pi / (n-1)
 
def L2_inn_trapz(f, g, dx):
  return np.trapz(f*g,dx=dx) / np.pi
 
def f(x):
    return 1
 
def g(x):
    return 2 * ( np.cos( 2*x ) )**(4)
 
#TEST inner product
test_res = L2_inn_trapz(f(X), g(X), dx)
print(test_res)
 
#TEST sqr norm
test_res = L2_inn_trapz(diff_func(f(X), g(X)), diff_func(f(X), g(X)), dx)
print(test_res**(1/2))
 
def test_int1( x_k, x_k1 ):
    return ( ( -1 * ( x_k1**3 - x_k**3 )        / 3 +
                    ( x_k1**2 - x_k**2 ) * x_k1 / 2 ) /
            ( x_k1 - x_k ) )
 
def test_int2( x_k, x_k1):
    return ((x_k1**3 - x_k**3) / 3 - x_k * (x_k1**2 - x_k**2) / 2)/((x_k1 - x_k))

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> (14) Вывод вычисленных данных на одной сетке </font></b>

print(data_to_output)